# Naxos Music Library


## Background

Based on a brief conversation with Mackenzie Miller (then UM Interactive Media grad student) in early 2019, I've decided to build a scraper to get data from the Naxos Music Library about classical music in movies. I built the scraper originally in early 2019 and now in October 2019, I'm updating the code so it can be reusable. This effort is also an example of how annoying it can be to try to scrape certain websites (I'll get more into this later). 

With Naxos, you can see the data organized by either [Movie Title](https://www.naxos.com/musicinmovieslist.asp?letter=A) or [Composer](https://www.naxos.com/musicinmoviescomplist.asp?letter=A). I've chosen to go with the Composer arrangement because that's how Mackenzie and I were considering the data at first, and it seems like there are less composers than movies (although I haven't verified this).


## The Plan

1. Get the HTML from each page and save it as a backup. Because no composer's name starts with a number (yet), I only need to scrape the 26 pages that correspond to the letters of the alphabet. There are a few letters for which there is no composer with that last name: N, O, U, and X. **I'm not exactly sure if classical music only refers to music made in a certain period or if there are composers today who do compose classical music. Because of this I'll scrape the pages now, in case there are composers added whose last name start with these letters.**
1. For each page, find the table inside the `<td>` element with class `style5` and then work to get the data from there. Save the data as a CSV file.

@@TODO: Figure out how to save the data a JSON file.


---

The first we'll do is import the various libraries we'll be working with:

* @@TODO: explain each of the libraries being used.
* @@TODO: add this info to the .README file

In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
from time import sleep
import csv
from string import ascii_uppercase
import codecs

The url structure for the composers whose last names start with the letter `A` is:

`https://www.naxos.com/musicinmoviescomplist.asp?letter=A`

If we want to get the composers whose last names start with the letter `B` we just have to change what comes after the `=` in the url. We can conveniently get the entire alphabet in uppercase as a string by using `ascii_uppercase` from the `string` module.

To get the html for the pages we'll loop through the string and request each page. We'll save the scraped HTML to a folder that will follow the pattern: `YYYY-MM-pages`. I'm doing this in October 2019, so the folder will be `2019-10-pages`. Currently, this is a manual process, but I hope to automate it.

* @@TODO: automate the month/year pattern for folder creation
* @@TODO: automate the folder creation in the notebook


In [2]:
base_url = "https://www.naxos.com/musicinmoviescomplist.asp?letter="
year = 2019
month = 10
out_folder = "-".join([str(year), str(month), "pages"])

We'll use the `requests` library to request the html code of each page and save it for processing. This part of the code requires an internet connection, but the rest of the notebook does not. We'll append the path to each file to a list that we'll use when working with our data. 

In [3]:
pages = []

for letter in ascii_uppercase:
    out_file = out_folder + "/" + letter + ".html"

    complete_url = base_url + letter
    response = requests.get(complete_url)
    page = response.text
    
    pages.append(out_file)
    open(out_file, "w+").write(page)

Before we process the 26 files, let's look at some examples so we can build out the overall scraper. The two big cases we have to write code for are:

1. When we have composer data on the page.
1. When we don't have composer data on the page. 

Let's start with the page for composers who have last names starting with the letter P.

In [4]:
case1_file = codecs.open(pages[15], 'r', 'utf-8')
case1 = case1_file.read()

With this first case, we'll identify any additional formatting and prepping of the file that may need to be done and then surf the html to find the fields we could want.

The first thing to note is that the files are encoded as `ISO-8859-1` and not `UTF-8`. Before I pass them on to Beautiful Soup for the parsing, I want them to be in `UTF-8`.

I'll also remove the invisible characters: `\n`, `\t`, and `\r` to make future parsing quicker

In [5]:
case1 = case1.replace("\n", "").replace("\t", "").replace("\r", "")

soup = BeautifulSoup(case1, "lxml")

After inspecting the website, I noticed that the table we want to scrape is inside a `<td>` element with a class attribute `style5`. The table that is a child of this `<td class="style5"></td>` element is what we care about. Let's get that first

In [6]:
table = soup.find("td", class_="style5").find("table")

Within this table, the composers are each within `<b></b>` tags. Let's grab those first.

In [7]:
composers = table.find_all("b")
composers

[<b>PACHELBEL, JOHANN</b>,
 <b>PAGANINI, NICOLO</b>,
 <b>PAISIELLO, GIOVANNI</b>,
 <b>PALESTRINA, GIOVANNI PIERLUIGI DA</b>,
 <b>PALSCHAU, JOHANN GOTTFRIED WILHELM</b>,
 <b>PARRY, HUBERT</b>,
 <b>PART, ARVO</b>,
 <b>PEARSALL, ROBERT LUCAS</b>,
 <b>PENDERECKI, KRZYSZTOF</b>,
 <b>PERGOLESI, GIOVANNI BATTISTA</b>,
 <b>PITONI, GIUSEPPE OTTAVIO</b>,
 <b>PONCHIELLI, AMILCARE</b>,
 <b>POULENC, FRANCIS</b>,
 <b>PROKOFIEV, SERGEY</b>,
 <b>PUCCINI, GIACOMO</b>,
 <b>PURCELL, HENRY</b>]

After the composer info, we want the meat of the data: the composer's music that was used, and the movie it was used in. This information is conveniently packaged in various `<div></div>` containers, but these div containers are in the next row. From the composer name, we have to climb out to the `<tr></tr>` level and then find the next row. Let's try it with the first composer in this list:

In [8]:
composer = composers[0]
composer_row = table.find(text=composer.text).parent.parent.parent
composer_row

<tr><td bgcolor="#EEEEEE"><b>PACHELBEL, JOHANN</b></td></tr>

Now we have to find the next row in the table.

In [9]:
music_row = composer_row.next_sibling.next_sibling
music_row

<tr><td bgcolor="#ffffff" class="style5"> <div style="border-top:1px dotted #999999; border-bottom:1px dotted #999999; padding:3px 0px; margin:0px 0px">Canon in D major <span class="style1">(<a href="https://www.naxos.com/catalogue/item.asp?item_code=8.550104">8.550104</a> )</span><br/><i>Brooklyn (2015) <br/> Dude, Where's My Car? (2000) <br/> Father of the Bride (1991) <br/> Hush <br/> Laggies (2014) <br/> Me Before You (2016) <br/> Nothing to Lose (1997) <br/> Ordinary People (1980) <br/> Queens Logic (1991) <br/> Runaway Bride (1999) <br/> Sing (2016) <br/> Zoolander (2001)</i><br/> </div> </td></tr>

There is a div for each piece of a composer that was used in a movie. Let's get them:

In [10]:
pieces_in_movies = music_row.find_all("div")
pieces_in_movies

[<div style="border-top:1px dotted #999999; border-bottom:1px dotted #999999; padding:3px 0px; margin:0px 0px">Canon in D major <span class="style1">(<a href="https://www.naxos.com/catalogue/item.asp?item_code=8.550104">8.550104</a> )</span><br/><i>Brooklyn (2015) <br/> Dude, Where's My Car? (2000) <br/> Father of the Bride (1991) <br/> Hush <br/> Laggies (2014) <br/> Me Before You (2016) <br/> Nothing to Lose (1997) <br/> Ordinary People (1980) <br/> Queens Logic (1991) <br/> Runaway Bride (1999) <br/> Sing (2016) <br/> Zoolander (2001)</i><br/> </div>]

We have to parse each one, but let's start with the first piece.

In [11]:
piece1 = pieces_in_movies[0]
piece1

<div style="border-top:1px dotted #999999; border-bottom:1px dotted #999999; padding:3px 0px; margin:0px 0px">Canon in D major <span class="style1">(<a href="https://www.naxos.com/catalogue/item.asp?item_code=8.550104">8.550104</a> )</span><br/><i>Brooklyn (2015) <br/> Dude, Where's My Car? (2000) <br/> Father of the Bride (1991) <br/> Hush <br/> Laggies (2014) <br/> Me Before You (2016) <br/> Nothing to Lose (1997) <br/> Ordinary People (1980) <br/> Queens Logic (1991) <br/> Runaway Bride (1999) <br/> Sing (2016) <br/> Zoolander (2001)</i><br/> </div>

For each time music was used we have a few fields:

* name of the piece
* link to the piece in the Naxos catalog. We can also get the item code from here.
* the movie(s) the piece was in. This field also currently contains the year the movie was produced, and we'll isolate that field when we clean the data later.

Our next goal is to identify each of these areas with some BeautifulSoup query. Let's try to find the name of the piece first

In [12]:
piece_name = piece1.find("span").previous_sibling
piece_name

'Canon in D major '

This is how we can get the name of the piece based on the html code we can access. Looking at this, we can strip the field to get rid of traling spaces before or after the title.

In [13]:
piece_name = piece1.find("span").previous_sibling.strip()
piece_name

'Canon in D major'

Now that we have the name, let's look for the piece's item code and url in the Naxos catalog.

In [14]:
piece_url = piece1.find("span").find("a")["href"]
piece_url

'https://www.naxos.com/catalogue/item.asp?item_code=8.550104'

In [15]:
piece_item_code = piece1.find("span").find("a").text
piece_item_code

'8.550104'

The last thing we need is the movies the piece is in. The movies are enclosed within `<i></i>` tags and if there are multiple movies for a piece (like Johann Pachelbel's Canon in D), there are `<br>` in the between the movies. I'll address the different movies when I get to tidying the data.

In [16]:
movie_list = piece1.find("i")
movie_list

<i>Brooklyn (2015) <br/> Dude, Where's My Car? (2000) <br/> Father of the Bride (1991) <br/> Hush <br/> Laggies (2014) <br/> Me Before You (2016) <br/> Nothing to Lose (1997) <br/> Ordinary People (1980) <br/> Queens Logic (1991) <br/> Runaway Bride (1999) <br/> Sing (2016) <br/> Zoolander (2001)</i>

Now that we figured out these steps, let's build things back up. And try to get all the info for the different composers on this page:

In [17]:
data_list = []

for composer in composers:
    name = composer.text
    composer_row = table.find(text=composer.text).parent.parent.parent
    music_row = composer_row.next_sibling.next_sibling
    pieces = music_row.find_all("div")
    for piece in pieces:
        piece_name = piece.find("span").previous_sibling.strip()
        piece_url = piece.find("span").find("a")["href"]
        piece_item_code = piece.find("span").find("a").text
        movie_list = piece.find("i").text.replace("  ","?;")
        data_list.append([name, piece_name, piece_url, piece_item_code, movie_list])
    

In [18]:
data_list

[['PACHELBEL, JOHANN',
  'Canon in D major',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.550104',
  '8.550104',
  "Brooklyn (2015)?;Dude, Where's My Car? (2000)?;Father of the Bride (1991)?;Hush?;Laggies (2014)?;Me Before You (2016)?;Nothing to Lose (1997)?;Ordinary People (1980)?;Queens Logic (1991)?;Runaway Bride (1999)?;Sing (2016)?;Zoolander (2001)"],
 ['PAGANINI, NICOLO',
  '24 Caprices, Op. 1: No. 20 in D major',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.550717',
  '8.550717',
  'Magic Bow (The) (1946)'],
 ['PAGANINI, NICOLO',
  'Centone di sonate, Op. 64, MS 112: Sonata No. 3 in C major',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.553141',
  '8.553141',
  'Gallipoli (1981)'],
 ['PAGANINI, NICOLO',
  'Violin Concertos Nos. 1 & 2',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.550649',
  '8.550649',
  'Magic Bow (The) (1946)'],
 ['PAISIELLO, GIOVANNI',
  'Il barbiere di Siviglia (The Barber of Seville): Saper bramate',
  'https://ww

This seems to work! Let's try it on a page with not table and see what we need to add. We'll start the same way:

In [19]:
case2= codecs.open(pages[13], 'r', 'utf-8').read()
case2 = case2.replace("\n", "").replace("\t", "").replace("\r", "")

soup2 = BeautifulSoup(case2, "lxml")

And now we'll look into finding the composers. I'm compressing things that we did more openly before.

In [20]:
soup2.find("td", class_="style5").find("table").find_all("b")

[]

By looking for the composers we get an empty table. We could build a checkpoint around the composer names. If we can find composers, proceed with the scraping, otherwise pass to the next page. But we can also take advantage of this neat thing with lists:


In [21]:
for i in []: 
    print(2)
print(1)

1


In [22]:
for item in []: 
    print("yes")

print("no")

no


If the list is empty, the inside of a for-loop will never get executed. So we don't really have to add anything.

Now we can move to extending what we did for the letter `P` to all the other letters.

In [23]:
untidy_data = [["composer", "piece", "piece_url", "piece_item_code", "movie_list"]]

for page in pages:
    file = codecs.open(page, "r", "utf-8").read().replace("\n", "").replace("\t", "").replace("\r", "")
    soup = BeautifulSoup(file, "lxml")
    
    table = soup.find("td", class_="style5").find("table")
    composers = table.find_all("b")
    for composer in composers:
        name = composer.text
        composer_row = table.find(text=composer.text).parent.parent.parent
        music_row = composer_row.next_sibling.next_sibling
        pieces = music_row.find_all("div")
        for piece in pieces:
            piece_name = piece.find("span").previous_sibling.strip()
            piece_url = piece.find("span").find("a")["href"]
            piece_item_code = piece.find("span").find("a").text
            movie_list = piece.find("i").text.replace("  ","?;")
            untidy_data.append([name, piece_name, piece_url, piece_item_code, movie_list])
    
print("done")

done


In [24]:
untidy_data

[['composer', 'piece', 'piece_url', 'piece_item_code', 'movie_list'],
 ['ADAM, ADOLPHE',
  'Giselle: Apparition de Giselle',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.550755-56',
  '8.550755-56',
  'Red Shoes (The) (1948)'],
 ['ADAM, ADOLPHE',
  "Giselle: Entree d'Hilarion, scene et fugue des Wilis",
  'https://www.naxos.com/catalogue/item.asp?item_code=8.550755-56',
  '8.550755-56',
  'Red Shoes (The) (1948)'],
 ['ADAM, ADOLPHE',
  'Giselle: Pas de deux des jeunes paysans',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.550755-56',
  '8.550755-56',
  'Red Shoes (The) (1948)'],
 ['ADAM, ADOLPHE',
  'Giselle: Pas des premieres Wilis',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.550755-56',
  '8.550755-56',
  'Red Shoes (The) (1948)'],
 ['ADAMS, JOHN',
  'China Gates',
  'https://www.naxos.com/catalogue/item.asp?item_code=8.559285',
  '8.559285',
  'Call Me by Your Name (2017)'],
 ['ADAMS, JOHN',
  'Hallelujah Junction',
  'https://www.naxos.com/catalog


We can now write this Python list to a CSV file:

In [25]:
out_data_file = "-".join([str(year), str(month), "untidy"])+".csv"
with open(out_data_file, "w") as file:
    csv.writer(file, delimiter=',').writerows(untidy_data)

The rest of the work will be in another notebook.